# T1055.004 - Asynchronous Procedure Call
Adversaries may inject malicious code into processes via the asynchronous procedure call (APC) queue in order to evade process-based defenses as well as possibly elevate privileges. APC injection is a method of executing arbitrary code in the address space of a separate live process. 

APC injection is commonly performed by attaching malicious code to the APC Queue (Citation: Microsoft APC) of a process's thread. Queued APC functions are executed when the thread enters an alterable state.(Citation: Microsoft APC) A handle to an existing victim process is first created with native Windows API calls such as <code>OpenThread</code>. At this point <code>QueueUserAPC</code> can be used to invoke a function (such as <code>LoadLibrayA</code> pointing to a malicious DLL). 

A variation of APC injection, dubbed "Early Bird injection", involves creating a suspended process in which malicious code can be written and executed before the process' entry point (and potentially subsequent anti-malware hooks) via an APC. (Citation: CyberBit Early Bird Apr 2018) AtomBombing (Citation: ENSIL AtomBombing Oct 2016) is another variation that utilizes APCs to invoke malicious code previously written to the global atom table.(Citation: Microsoft Atom Table)

Running code in the context of another process may allow access to the process's memory, system/network resources, and possibly elevated privileges. Execution via APC injection may also evade detection from security products since the execution is masked under a legitimate process. 

## Atomic Tests

### Atomic Test #1 - Process Injection via C#
Process Injection using C#
reference: https://github.com/pwndizzle/c-sharp-memory-injection
Excercises Five Techniques
1. Process injection
2. ApcInjectionAnyProcess
3. ApcInjectionNewProcess
4. IatInjection
5. ThreadHijack
Upon successful execution, cmd.exe will execute T1055.exe, which exercises 5 techniques. Output will be via stdout.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: #{exe_binary} must be exist on system.

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1055.004\bin\T1055.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1055.004\bin\T1055.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1055.004/bin/T1055.exe" -OutFile "PathToAtomicsFolder\T1055.004\bin\T1055.exe"

```

In [ ]:
Invoke-AtomicTest T1055.004 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\T1055.004\bin\T1055.exe"
```

In [ ]:
Invoke-AtomicTest T1055.004 -TestNumbers 1

### Atomic Test #2 - EarlyBird APC Queue Injection in GoCreates a process in a suspended state and calls QueueUserAPC WinAPI to add a UserAPC to the child process that points to allocated shellcode. 
ResumeThread is called which then calls NtTestAlert to execute the created UserAPC which then executes the shellcode.
This technique allows for the early execution of shellcode and potentially before AV/EDR can hook functions to support detection.
- PoC Credit: (https://github.com/Ne0nd0g/go-shellcode#createprocesswithpipe)
- References: 
  - https://www.bleepingcomputer.com/news/security/early-bird-code-injection-technique-helps-malware-stay-undetected/
  - https://www.ired.team/offensive-security/code-injection-process-injection/early-bird-apc-queue-code-injection
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$PathToAtomicsFolder\T1055.004\bin\x64\EarlyBird.exe -program "C:\Windows\System32\werfault.exe" -debug
```

In [ ]:
Invoke-AtomicTest T1055.004 -TestNumbers 2

#### Cleanup: 
```powershell
Stop-Process -Name CalculatorApp -ErrorAction SilentlyContinue
Stop-Process -Name "werfault" -ErrorAction SilentlyContinue
```

In [ ]:
Invoke-AtomicTest T1055.004 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Remote Process Injection with Go using NtQueueApcThreadEx WinAPIUses the undocumented NtQueueAPCThreadEx WinAPI to create a "Special User APC" in the current thread of the current process to execute shellcode. 
Since the shellcode is loaded and executed in the current process it is considered local shellcode execution.

Steps taken with this technique
1. Allocate memory for the shellcode with VirtualAlloc setting the page permissions to Read/Write
2. Use the RtlCopyMemory macro to copy the shellcode to the allocated memory space
3. Change the memory page permissions to Execute/Read with VirtualProtect
4. Get a handle to the current thread
5. Execute the shellcode in the current thread by creating a Special User APC through the NtQueueApcThreadEx function

- PoC Credit: (https://github.com/Ne0nd0g/go-shellcode/tree/master#rtlcreateuserthread)
- References:
  - https://repnz.github.io/posts/apc/user-apc/
  - https://docs.rs/ntapi/0.3.1/ntapi/ntpsapi/fn.NtQueueApcThreadEx.html
  - https://0x00sec.org/t/process-injection-apc-injection/24608
  - https://twitter.com/aionescu/status/992264290924032005
  - http://www.opening-windows.com/techart_windows_vista_apc_internals2.htm#_Toc229652505
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$PathToAtomicsFolder\T1055.004\bin\x64\NtQueueApcThreadEx.exe -debug
```

In [ ]:
Invoke-AtomicTest T1055.004 -TestNumbers 3

#### Cleanup: 
```powershell
Stop-Process -Name CalculatorApp -ErrorAction SilentlyContinue```

In [ ]:
Invoke-AtomicTest T1055.004 -TestNumbers 3 -Cleanup

## Detection
Monitoring Windows API calls indicative of the various types of code injection may generate a significant amount of data and may not be directly useful for defense unless collected under specific circumstances for known bad sequences of calls, since benign use of API functions may be common and difficult to distinguish from malicious behavior. Windows API calls such as <code>SuspendThread</code>/<code>SetThreadContext</code>/<code>ResumeThread</code>, <code>QueueUserAPC</code>/<code>NtQueueApcThread</code>, and those that can be used to modify memory within another process, such as <code>VirtualAllocEx</code>/<code>WriteProcessMemory</code>, may be used for this technique.(Citation: Elastic Process Injection July 2017)

Analyze process behavior to determine if a process is performing actions it usually does not, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior. 